In [2]:
import sqlite3
import tkinter as tk
from tkinter import messagebox, ttk

# Create the database and tables if they don't exist
def create_db():
    conn = sqlite3.connect('prompts.db')
    c = conn.cursor()
    
    c.execute('''
    CREATE TABLE IF NOT EXISTS prompts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        content TEXT NOT NULL
    )
    ''')

    c.execute('''
    CREATE TABLE IF NOT EXISTS submitted_prompts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        content TEXT NOT NULL
    )
    ''')
    
    conn.commit()
    conn.close()

# Function to remove duplicates from prompts table
def remove_duplicates():
    conn = sqlite3.connect('prompts.db')
    c = conn.cursor()
    
    c.execute('''
        DELETE FROM prompts
        WHERE rowid NOT IN (
            SELECT MAX(rowid)
            FROM prompts
            GROUP BY title, content
        )
    ''')
    conn.commit()
    conn.close()

# Function to recreate prompts table to reset ID sequence
def recreate_table():
    conn = sqlite3.connect('prompts.db')
    c = conn.cursor()
    
    c.execute('''
    CREATE TABLE IF NOT EXISTS new_prompts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        content TEXT NOT NULL
    )
    ''')
    
    c.execute('''
    INSERT INTO new_prompts (title, content)
    SELECT title, content FROM prompts
    ''')
    
    c.execute('DROP TABLE prompts')
    
    c.execute('ALTER TABLE new_prompts RENAME TO prompts')
    
    conn.commit()
    conn.close()

# Function to approve a submitted prompt
def approve_prompt(selected_prompt):
    conn = sqlite3.connect('prompts.db')
    c = conn.cursor()
    
    c.execute('INSERT INTO prompts (title, content) VALUES (?, ?)', (selected_prompt[1], selected_prompt[2]))
    c.execute('DELETE FROM submitted_prompts WHERE id = ?', (selected_prompt[0],))
    
    conn.commit()
    conn.close()
    refresh_tables()

# Function to reject a submitted prompt
def reject_prompt(selected_prompt):
    conn = sqlite3.connect('prompts.db')
    c = conn.cursor()
    
    c.execute('DELETE FROM submitted_prompts WHERE id = ?', (selected_prompt[0],))
    
    conn.commit()
    conn.close()
    refresh_tables()

# Function to refresh the data in the tables
def refresh_tables():
    for row in tree_prompts.get_children():
        tree_prompts.delete(row)
    
    conn = sqlite3.connect('prompts.db')
    c = conn.cursor()
    c.execute('SELECT * FROM prompts')
    prompts = c.fetchall()
    for prompt in prompts:
        tree_prompts.insert('', 'end', values=prompt)
    
    conn.close()
    
    for row in tree_submitted_prompts.get_children():
        tree_submitted_prompts.delete(row)
    
    conn = sqlite3.connect('prompts.db')
    c = conn.cursor()
    c.execute('SELECT * FROM submitted_prompts')
    submitted_prompts = c.fetchall()
    for submitted_prompt in submitted_prompts:
        tree_submitted_prompts.insert('', 'end', values=submitted_prompt)
    
    conn.close()

# Function to handle approval button click
def approve_selected_prompt():
    selected_item = tree_submitted_prompts.selection()
    if selected_item:
        selected_prompt = tree_submitted_prompts.item(selected_item, 'values')
        approve_prompt(selected_prompt)
    else:
        messagebox.showwarning("No selection", "No prompt selected for approval")

# Function to handle reject button click
def reject_selected_prompt():
    selected_item = tree_submitted_prompts.selection()
    if selected_item:
        selected_prompt = tree_submitted_prompts.item(selected_item, 'values')
        reject_prompt(selected_prompt)
    else:
        messagebox.showwarning("No selection", "No prompt selected for rejection")

# Initialize the GUI
root = tk.Tk()
root.title("Prompt Management")

frame = ttk.Frame(root, padding="10")
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Prompts table
ttk.Label(frame, text="Prompts Table").grid(row=0, column=0, columnspan=2)
tree_prompts = ttk.Treeview(frame, columns=('id', 'title', 'content'), show='headings')
tree_prompts.heading('id', text='ID')
tree_prompts.heading('title', text='Title')
tree_prompts.heading('content', text='Content')
tree_prompts.grid(row=1, column=0, columnspan=2, sticky=(tk.W, tk.E, tk.N, tk.S))

# Submitted prompts table
ttk.Label(frame, text="Submitted Prompts Table").grid(row=2, column=0, columnspan=2)
tree_submitted_prompts = ttk.Treeview(frame, columns=('id', 'title', 'content'), show='headings')
tree_submitted_prompts.heading('id', text='ID')
tree_submitted_prompts.heading('title', text='Title')
tree_submitted_prompts.heading('content', text='Content')
tree_submitted_prompts.grid(row=3, column=0, columnspan=2, sticky=(tk.W, tk.E, tk.N, tk.S))

# Approval and rejection buttons
approve_button = ttk.Button(frame, text="Approve Selected Prompt", command=approve_selected_prompt)
approve_button.grid(row=4, column=0)

reject_button = ttk.Button(frame, text="Reject Selected Prompt", command=reject_selected_prompt)
reject_button.grid(row=4, column=1)

# Configure grid to expand
frame.columnconfigure(0, weight=1)
frame.rowconfigure(1, weight=1)
frame.rowconfigure(3, weight=1)

# Initialize database and populate tables
create_db()
remove_duplicates()
recreate_table()
refresh_tables()

# Run the GUI event loop
root.mainloop()
